In [ ]:
import os
import sys
import pandas as pd
import math
import matplotlib.pyplot as plt
import sweetviz as sv
import numpy as np
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, ParameterGrid, KFold
from sklearn.metrics import roc_auc_score, roc_curve, r2_score
from scipy.stats import spearmanr, pearsonr

In [ ]:
os.chdir('./scripts')

import load_config

In [ ]:
config = load_config.load_config_const()
plt.style.use('seaborn')

In [ ]:
df_dgt = load_config.obtain_dgt_dataset(config, 0.3)
df_dgt.sample(10)

In [ ]:
temporal_df_dgt = df_dgt.groupby(["FEC_TRAMITE"]).size().reset_index(name='count')

In [ ]:
temporal_df_dgt.sort_values('FEC_TRAMITE', ascending=True)

In [ ]:
temporal_df_dgt['date'] = temporal_df_dgt['FEC_TRAMITE']
temporal_df_dgt

In [ ]:
temporal_df_dgt = temporal_df_dgt.set_index('FEC_TRAMITE')

In [ ]:
temporal_df_dgt

In [ ]:
idx = pd.date_range(start = temporal_df_dgt["date"].min(), end = temporal_df_dgt["date"].max() , freq="d").difference(temporal_df_dgt["date"])

In [ ]:
idx

In [ ]:
temporal_df_dgt_no_data = pd.DataFrame({'FEC_TRAMITE': idx})
temporal_df_dgt_no_data

In [ ]:
temporal_df_dgt_no_data['date'] = temporal_df_dgt_no_data['FEC_TRAMITE']
temporal_df_dgt_no_data

In [ ]:
temporal_df_dgt_no_data['count'] = 0
temporal_df_dgt_no_data

In [ ]:
temporal_df_dgt_no_data = temporal_df_dgt_no_data.set_index('FEC_TRAMITE')
temporal_df_dgt_no_data

In [ ]:
frames = [temporal_df_dgt, temporal_df_dgt_no_data]

result_dgt = pd.concat(frames)

In [ ]:
result_dgt.sort_values('date', ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

sns.lineplot(data=result_dgt, x='date', y='count', ax=ax)

In [ ]:
def is_weekend(day_week):
    if (day_week == 'Saturday' or day_week == 'Sunday'):
        return 1.0
    else:
        return 0.0
    return False

In [ ]:
result_dgt['is_weekend'] = result_dgt['date'].dt.day_name().apply(lambda x: is_weekend(x))

In [ ]:
result_dgt.sort_values('date', ascending=True)

In [ ]:
x = np.arange(len(result_dgt))

result_dgt["s_period_week"] = np.sin(2*np.pi*x/7)
result_dgt["s_period_month"] = np.sin(2*np.pi*x/30.5)
result_dgt["s_period_year"] = np.sin(2*np.pi*x/365 )

result_dgt["c_period_week"] = np.cos(2*np.pi*x/7)
result_dgt["c_period_month"] = np.cos(2*np.pi*x/30.5)
result_dgt["c_period_year"] = np.cos(2*np.pi*x/365 )

In [ ]:
result_dgt.sort_values('date', ascending=True)

In [ ]:
result_dgt[(result_dgt['count'] == 0) & (result_dgt['is_weekend'] == 1.0)]['count']

In [ ]:
result_dgt[(result_dgt['count'] != 0) & (result_dgt['is_weekend'] == 1.0)]['count'].mean()

In [ ]:
result_dgt[(result_dgt['count'] == 0) & (result_dgt['is_weekend'] == 0.0)]['count']

In [ ]:
result_dgt[(result_dgt['count'] != 0) & (result_dgt['is_weekend'] == 0.0)]['count'].mean()

In [ ]:
result_dgt.loc[(result_dgt['count'] == 0) & (result_dgt['is_weekend'] == 1.0),'count'] = result_dgt[(result_dgt['count'] != 0) & (result_dgt['is_weekend'] == 1.0)]['count'].mean()

In [ ]:
result_dgt.loc[(result_dgt['count'] == 0) & (result_dgt['is_weekend'] == 0.0),'count'] = result_dgt[(result_dgt['count'] != 0) & (result_dgt['is_weekend'] == 0.0)]['count'].mean()

In [ ]:
result_dgt[(result_dgt['count'] == 0) & (result_dgt['is_weekend'] == 1.0)]

In [ ]:
result_dgt[(result_dgt['count'] == 0) & (result_dgt['is_weekend'] == 0.0)]

In [ ]:
result_dgt[result_dgt['date'] == '2015-02-21']

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

sns.lineplot(data=result_dgt, x='date', y='count', ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

sns.lineplot(data=result_dgt[result_dgt['date'] >= '2021-08-01' ], x='date', y='count', ax=ax)

In [ ]:
result_dgt.dtypes

In [ ]:
result_dgt.columns

In [ ]:
X = result_dgt[[
    'date', 'is_weekend', 's_period_week', 's_period_week', 's_period_year', 'c_period_week', 'c_period_week', 'c_period_year'
]]

y = result_dgt[['date','count']]

In [ ]:
X

In [ ]:
# Hacemos las pruebas con los últimos 4 meses
train_X = np.arange(len(X.index))[:-4*30]
test_X = np.arange(len(X.index))[-4*30:]

train_y = np.arange(len(y.index))[:-4*30]
test_y = np.arange(len(y.index))[-4*30:]

In [ ]:
train_X = X[X['date'] < '2021-09-01']
test_X = X[X['date'] >= '2021-09-01']

train_y = y[y['date'] < '2021-09-01']
test_y = y[y['date'] >= '2021-09-01']

In [ ]:
X = X[['is_weekend', 's_period_week', 's_period_week', 's_period_year', 'c_period_week', 'c_period_week', 'c_period_year']]
y = y['count']

train_X = train_X[['is_weekend', 's_period_week', 's_period_week', 's_period_year', 'c_period_week', 'c_period_week', 'c_period_year']]
test_X = test_X[['is_weekend', 's_period_week', 's_period_week', 's_period_year', 'c_period_week', 'c_period_week', 'c_period_year']]

train_y = train_y['count']
test_y = test_y['count']

In [ ]:
?RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators=600, min_samples_split=5, oob_score=True, max_depth=5)
rf.fit(train_X, train_y)

In [ ]:
rf.oob_score_

In [ ]:
rf.predict(train_X)

In [ ]:
y_hat = rf.predict(test_X)

test_score = r2_score(test_y, y_hat)


print(f'Out-of-bag R-2 score estimate: {rf.oob_score_:>5.3}')
print(f'Test data R-2 score: {test_score:>5.3}')

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(X=train_X,y=train_y, estimator=rf )

In [ ]:

grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 4, 6, 8, 10],
    'max_leaf_nodes': [5,10,25,50,100]
}


grid_search = GridSearchCV(
    RandomForestRegressor(),
    param_grid=grid,
    scoring="neg_mean_squared_error",
    verbose=2
)

In [ ]:
grid_search.fit(train_X,train_y)

In [ ]:
grid_search.best_params_

In [ ]:
pd.DataFrame(grid_search.cv_results_)

In [ ]:
rf_final = RandomForestRegressor(n_estimators=200, min_samples_split=6, oob_score=True, max_depth=5)
rf_final.fit(train_X, train_y)

In [ ]:
rf_final.oob_score_

In [ ]:
rf_final.predict(train_X)

In [ ]:
y_hat = rf.predict(test_X)

test_score = r2_score(test_y, y_hat)


print(f'Out-of-bag R-2 score estimate: {rf.oob_score_:>5.3}')
print(f'Test data R-2 score: {test_score:>5.3}')

In [ ]:
len(grid_search.predict(test_X))

In [ ]:
len(test_X.index)

In [ ]:
df_graph_final = test_X.copy()

In [ ]:
df_graph_final.sample(10)

In [ ]:
df_graph_final['date'] = df_graph_final.index

In [ ]:
df_graph_final.sample(10)

In [ ]:
df_graph_final['count'] = grid_search.predict(test_X)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

sns.lineplot(data=result_dgt[result_dgt['date'] >= '2021-08-01' ], x='date', y='count', ax=ax, color='blue', label='Datos reales')
sns.lineplot(data=df_graph_final, x='date', y='count', ax=ax, color='red', label='Predicción')